In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import pytorch_lightning
from nlp_datasets import get_dataloader, get_dataset

import torch
from torch import optim
from torch.optim.lr_scheduler import StepLR
from tensorboardX import SummaryWriter
from accelerate import Accelerator
from safetensors.torch import load_file

import json
import gc

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
writer = SummaryWriter()
accelerator = Accelerator()

In [3]:
torch.cuda.empty_cache()
gc.collect()

60

In [4]:
text_decoder_id = f"MLP-KTLim/llama-3-Korean-Bllossom-8B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(
    text_decoder_id,
    lagacy = False
)

tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
base_model = AutoModelForCausalLM.from_pretrained(
    text_decoder_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True, 
    quantization_config=bnb_config
    )

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
for name, module in base_model.named_modules():
    print(name)
    break

In [7]:
train_dataloader = get_dataloader()
valid_dataloader = get_dataloader(split='valid')

dataset_dict:  {'prompt': '\n        system : 한국말로만 대답하고 최대한 간결하고 알기쉽게 정리해줘.\n        user : 사건의 title과 판시사항을 보고 판결 결과와 그 이유를 예측해줘\n        \n title:[공직선거법위반]〈당내경선운동에 관한 공직선거법위반죄에 있어서 당내경선의 의미, 공소시효의 기산일, 공직선거법상 선거에 관한 여론조사 규정의 규율대상 범위에 관한 사건〉 [공2019하,2298]\n판시사항:[1] 정당이 당원과 당원이 아닌 자에게 경선후보자 중 누가 선거의 후보자가 되어야 하는지에 관한 선택의 의사를 표시하게 하는 당내경선이 공직선거법 제57조의3 제1항에서 정한 ‘당원과 당원이 아닌 자에게 투표권을 부여하여 실시하는 당내경선’에 해당하는지 여부(적극) 및 이때 투표권을 행사하는 방식은 반드시 투표용지에 기표하는 방법으로 제한되는지 여부(소극)와 여론조사 방식을 통하여 위와 같은 선택의 의사표시를 하도록 하는 방법도 포함하는지 여부(원칙적 적극)\n[2] 공직선거법 제268조 제1항 본문에서 말하는 단기 공소시효 기산일인 ‘당해 선거일’의 의미(=선거범죄와 직접 관련된 공직선거의 투표일) 및 이때 선거범죄가 당내경선운동에 관한 공직선거법 위반죄인 경우, 그에 대한 공소시효 기산일(=선거범죄와 직접 관련된 공직선거의 투표일)\n[3] 공직선거법 제108조 제5항, 제11항에서 정한 ‘선거에 관한 여론조사’에 ‘당내경선과 관련된 여론조사’도 포함되는지 여부(적극)\n[4] 공직선거법 제230조 제7항 제2호에서 말하는 ‘경선운동관계자’의 의미\n\n        assistant : \n    ', 'label': '\n그러므로 원심판결을 파기하고, 사건을 다시 심리·판단하도록 원심법원에 환송하기로 하여, 관여 대법관의 일치된 의견으로 주문과 같이 판결한다.\n상고이유를 판단한다.\n1. 검사의 상고이유에 관한 판단\n가. 공직선거법 제57조의2는 정당은 공직선거후보자를 추천하기 위하여

Map:   0%|          | 0/789 [00:00<?, ? examples/s]

dataset_dict:  {'prompt': '\n        system : 한국말로만 대답하고 최대한 간결하고 알기쉽게 정리해줘.\n        user : 사건의 title과 판시사항을 보고 판결 결과와 그 이유를 예측해줘\n        \n title:[직권남용권리행사방해ㆍ강요ㆍ국회에서의증언ㆍ감정등에관한법률위반]〈문화예술계 지원배제 등 관련 직권남용권리행사방해 사건〉 [공2020상,545]\n판시사항:[1] 대통령비서실장인 피고인이 대통령의 뜻에 따라 정무수석비서관실과 교육문화수석비서관실 등 수석비서관실과 문화체육관광부에 문화예술진흥기금 등 정부의 지원을 신청한 개인·단체의 이념적 성향이나 정치적 견해 등을 이유로 한국문화예술위원회·영화진흥위원회·한국출판문화산업진흥원이 수행한 각종 사업에서 이른바 좌파 등에 대한 지원배제를 지시하였다는 직권남용권리행사방해의 공소사실로 기소되었는데, 특별검사가 검찰을 통하여 또는 직접 청와대로부터 넘겨받아 원심에 제출한 ‘청와대 문건’의 증거능력이 문제 된 사안에서, 위 ‘청와대 문건’은 위법수집증거가 아니므로 증거능력이 있다고 본 원심판단을 수긍한 사례\n[2] 직권남용권리행사방해죄에서 말하는 ‘직권남용’의 의미 및 남용에 해당하는지 판단하는 기준 / 직권남용권리행사방해죄는 공무원이 직권을 남용하는 행위를 하였다는 것 외에 현실적으로 다른 사람에게 법령상 의무 없는 일을 하게 하였거나 다른 사람의 구체적인 권리행사를 방해하는 결과가 발생하여야 성립하는지 여부(적극) 및 ‘의무 없는 일을 하게 한 때’에 해당하는지 판단하는 기준 / 공무원이 직권을 남용하여 상대방인 공무원 또는 유관기관의 임직원으로 하여금 어떠한 일을 하게 하였는데, 상대방이 한 일이 형식과 내용 등에 있어 직무범위 내에 속하는 사항으로서 법령 그 밖의 관련 규정에 따라 직무수행 과정에서 준수하여야 할 원칙이나 기준, 절차 등을 위반하지 않는 경우, 법령상 의무 없는 일을 하게 한 때에 해당하는지 여부(원칙적 소극)\n[3] 대통령비서실장을 비롯한

Map:   0%|          | 0/88 [00:00<?, ? examples/s]

In [8]:
## ---  LoRA --- ##
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    # modules_to_save=["q_proj", "k_proj", "v_proj"],
)
model = get_peft_model(base_model, config)
model.print_trainable_parameters()

peft_model_id = f"MLP-KTLim/llama-3-Korean-Bllossom-8B_Lora"

trainable params: 4,718,592 || all params: 8,172,867,584 || trainable%: 0.0577


In [9]:
raise ValueError()

ValueError: 

In [10]:
num_epochs = 20
learning_rate = 2e-5
optimizer = optim.Adam(
    model.parameters(),
    lr=learning_rate,
)
lr_scheduler = StepLR(optimizer, step_size=1, gamma=0.85)

In [11]:
# accelerator.prepare(
#     train_dataloader, valid_dataloader, model, optimizer, lr_scheduler
# )

In [12]:
def save_to_json(output_filename, epoch, train_epoch_loss, train_epoch_ppl, val_epoch_loss, val_epoch_ppl):
    metrics_data = {
        "epoch" : epoch,
        "train_epoch_loss": train_epoch_loss,
        "train_epoch_perplexity": train_epoch_ppl,
        "val_epoch_loss": val_epoch_loss,
        "val_epoch_perplexity": val_epoch_ppl
    }
    with open(output_filename, "a+") as f:
        json.dump(metrics_data, f)
        f.write('\n')


In [13]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [14]:
from tqdm import tqdm

device = "cuda"
model = model.to(device)
scaler = torch.cuda.amp.GradScaler()

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        # print(batch)
        with torch.autocast(device_type='cuda'):
            outputs = model(**batch)
            loss = outputs.loss
            total_loss += loss.detach().float()
        
        # accelerator.backward(loss)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        # loss.backward()
        # optimizer.step()
        # lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(valid_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )
    

    eval_epoch_loss = eval_loss / len(valid_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")
    save_to_json("./finetuning_train_result.json", epoch, float(train_epoch_loss), float(train_ppl.detach().cpu()), float(eval_epoch_loss), float(eval_ppl.detach().cpu()))
    

  0%|          | 0/789 [00:00<?, ?it/s]

100%|██████████| 88/88 [00:27<00:00,  3.18it/s]


epoch=0: train_ppl=tensor(21.3316, device='cuda:0') train_epoch_loss=tensor(3.0602, device='cuda:0') eval_ppl=tensor(13.3448, device='cuda:0') eval_epoch_loss=tensor(2.5911, device='cuda:0')


100%|██████████| 88/88 [00:27<00:00,  3.18it/s]


epoch=1: train_ppl=tensor(12.3089, device='cuda:0') train_epoch_loss=tensor(2.5103, device='cuda:0') eval_ppl=tensor(11.4716, device='cuda:0') eval_epoch_loss=tensor(2.4399, device='cuda:0')


...

epoch=17: train_ppl=tensor(6.2827, device='cuda:0') train_epoch_loss=tensor(1.8378, device='cuda:0') eval_ppl=tensor(9.9122, device='cuda:0') eval_epoch_loss=tensor(2.2938, device='cuda:0')


100%|██████████| 88/88 [00:27<00:00,  3.18it/s]


epoch=18: train_ppl=tensor(6.1300, device='cuda:0') train_epoch_loss=tensor(1.8132, device='cuda:0') eval_ppl=tensor(10.2014, device='cuda:0') eval_epoch_loss=tensor(2.3225, device='cuda:0')


100%|██████████| 88/88 [00:27<00:00,  3.18it/s]

epoch=19: train_ppl=tensor(5.9825, device='cuda:0') train_epoch_loss=tensor(1.7888, device='cuda:0') eval_ppl=tensor(10.2149, device='cuda:0') eval_epoch_loss=tensor(2.3238, device='cuda:0')


In [1]:
model.save_pretrained(f"./{peft_model_id}/")

NameError: name 'model' is not defined

In [9]:
from peft import PeftModel

text_decoder_id = f"MLP-KTLim/llama-3-Korean-Bllossom-8B"
peft_model_id = f"MLP-KTLim/llama-3-Korean-Bllossom-8B_Lora"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# LoRA 적용 모델 불러오기
base_model = AutoModelForCausalLM.from_pretrained(
    text_decoder_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True, 
    quantization_config=bnb_config
    )


load_model = PeftModel.from_pretrained(base_model, peft_model_id)
loaded_state_dict = load_file("/home/eahc00/NLP/term_project/MLP-KTLim/llama-3-Korean-Bllossom-8B_Lora/adapter_model.safetensors")
load_model.load_state_dict(loaded_state_dict, strict=False)
tokenizer = AutoTokenizer.from_pretrained(text_decoder_id)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
ds = get_dataset(split="valid", get_prompt_only=True)

# load_model.to('cuda')

i = 30
# inputs = tokenizer(ds[i]['prompt'], return_tensors="pt")
print(ds[i]['prompt'])
print(ds[i]['label'])


        system : 한국말로만 대답하고 최대한 간결하고 알기쉽게 정리해줘.
        user : 사건의 title과 판시사항을 보고 판결 결과와 그 이유를 예측해줘
        
 title:[시정명령및과징금납부명령취소]〈대규모유통업에서의 거래 공정화에 관한 법률 위반행위 사건〉 [공2020하,1110]
판시사항:대규모유통업에서의 거래 공정화에 관한 법률 제11조 제5항에서 납품업자 등의 요청이 ‘자발적’이라는 것의 의미 및 판매촉진행사가 다른 납품업자 등과 ‘차별화’된다고 하기 위한 요건 / 위 조항에서 정한 판매촉진행사인지 판단할 때 자발성과 차별성의 상관관계 / 자발성과 차별성을 판단하는 방법 및 그에 관한 증명책임의 소재(=대규모유통업자)

        assistant : 
    

그러므로 원심판결을 파기하고, 사건을 다시 심리·판단하게 하기 위하여 원심법원에 환송하기로 하여, 관여 대법관의 일치된 의견으로 주문과 같이 판결한다.
상고이유를 판단한다.
1. 관련 규정과 법리
가. 대규모유통업에서의 거래 공정화에 관한 법률(이하 ‘대규모유통업법’이라고 한다) 제11조에 의하면, 대규모유통업자는 판매촉진행사를 실시하기 이전에 대통령령으로 정하는 바에 따라 판매촉진행사에 소요되는 비용(이하 ‘판매촉진비용’이라고 한다)의 부담 등을 납품업자 등(납품업자 또는 매장임차인을 말한다, 이하 같다)과 약정하지 아니하고는 이를 납품업자 등에게 부담시키는 행위를 하여서는 아니 된다(제1항). 이러한 사전 약정은 대규모유통업자와 납품업자 등이 각각 서명 또는 기명날인한 서면으로 이루어져야 하며, 대규모유통업자는 약정과 동시에 그 서면을 납품업자 등에게 주어야 한다(제2항). 약정에 따른 판매촉진비용의 분담비율은 대규모유통업자와 납품업자 등이 각각 해당 판매촉진행사를 통하여 직접적으로 얻을 것으로 예상되는 경제적 이익의 비율에 따라 정하되(제3항), 납품업자 등의 판매촉진비용 분담비율은 100분의 50을 초과하여서는 아니 된다(제4항). 이는 대규

In [7]:
device = "cuda"

with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = load_model.generate(input_ids=inputs["input_ids"], max_new_tokens=512)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:144783 for open-end generation.


['\n        system : 한국말로만 대답하고 최대한 간결하고 알기쉽게 정리해줘.\n        user : 사건의 title과 판시사항을 보고 판결 결과와 그 이유를 예측해줘\n        \n title:[우선재고용의무위반등]〈경영상 이유에 의해 해고된 근로자가 우선 재고용 의무의 이행과 손해배상 등을 청구한 사건〉 [공2021상,89]\n판시사항:[1] 사용자가 근로기준법 제24조에 따라 근로자를 해고한 날부터 3년 이내의 기간 중에 해고 근로자가 해고 당시에 담당하였던 업무와 같은 업무를 할 근로자를 채용하려고 하는 경우, 해고 근로자를 우선 재고용할 의무가 있는지 여부(원칙적 적극) 및 이때 사용자가 해고 근로자에게 고용계약을 체결할 의사가 있는지 확인하지 않은 채 제3자를 채용한 경우, 근로기준법 제25조 제1항에서 정한 우선 재고용의무를 위반한 것인지 여부(원칙적 적극)\n[2] 갑이 을 재단법인이 운영하는 장애인 복지시설에서 생활부업무 담당 생활재활교사로 근무하다가 경영상 이유에 의하여 해고된 후 3년 이내의 기간 중에 을 법인이 여러 차례 생활재활교사를 채용하면서 갑에게 채용 사실을 고지하거나 고용계약을 체결할 의사가 있는지 확인하지 아니하였는데, 을 법인이 근로기준법 제25조 제1항에서 정한 우선 재고용의무를 위반한 시점이 문제 된 사안에서, 늦어도 갑이 해고 당시 담당하였던 생활부업무 담당 생활재활교사 업무에 근로자를 2명째 채용한 무렵에는 을 법인의 우선 재고용의무가 발생하였다고 한 사례\n[3] 사용자가 근로기준법 제25조 제1항에서 정한 우선 재고용의무를 이행하지 아니하는 경우, 해고 근로자가 사용자를 상대로 고용의 의사표시를 갈음하는 판결을 구할 사법상의 권리가 있는지 여부(적극) 및 판결이 확정되면 사용자와 해고 근로자 사이에 고용관계가 성립하는지 여부(적극) / 이때 해고 근로자가 사용자의 우선 재고용의무 불이행에 대하여 우선 재고용의무가 발생한 때부터 고용관계가 성립할 때까지의 임금 상당 손해배상금을 청구할 수 있

In [6]:
def save_to_json_rouge(output_filename, valid_idx, r1, r2, rl, r1_recall, r2_recall, rl_recall):
    metrics_data = {
        "valid_idx" : valid_idx,
        "rouge1": r1,
        "rouge2": r2,
        "rougel": rl,
        "rouge1_recall": r1_recall,
        "rouge2_recall": r2_recall,
        "rougel_recall" : rl_recall
    }
    with open(output_filename, "a+") as f:
        json.dump(metrics_data, f)
        f.write('\n')


In [7]:
from rouge_score import rouge_scorer
def get_rouge(tgt, hyp):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(tgt, hyp)
    r1 = scores["rouge1"].fmeasure
    recall_1 = scores["rouge1"].recall
    r2 = scores["rouge2"].fmeasure
    recall_2 = scores["rouge2"].recall
    rl = scores["rougeL"].fmeasure
    recall_l = scores["rougeL"].recall
    return r1, r2, rl, recall_1, recall_2, recall_l

In [12]:
ds = get_dataset(split="valid", get_prompt_only=True)
device = "cuda"

load_model.to(device)

for i in range(len(ds)) :
    inputs = tokenizer(ds[i]['prompt'], return_tensors="pt")

    with torch.no_grad():
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = load_model.generate(input_ids=inputs["input_ids"], max_new_tokens=512)
        generated_text = str(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))
    r1, r2, rl, recall_1, recall_2, recall_l = get_rouge(ds[i]['label'], generated_text)
    
    save_to_json_rouge("./finetuning_eval_result_rouge.json", i, r1, r2, rl, recall_1, recall_2, recall_l)

# print(ds[i]['prompt'])
# print(ds[i]['label'])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:144783 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:144783 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:144783 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:144783 for open-end gene

: 